In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


import tensorflow as tf
from keras.models import Sequential,load_model
from keras import layers,Input
from keras.losses import binary_crossentropy
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import L1L2
import keras.backend as K

2023-11-06 10:13:05.315569: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-06 10:13:07.227900: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
state0=np.load('../Data/new/2gsps/state0_her.npy')
state1=np.load('../Data/new/2gsps/state1_her.npy')
output0=np.zeros((state0.shape[0]))
output1=np.ones((state1.shape[0]))
print(state0.shape)
x=np.vstack((state0,state1))
print(x.shape)
y=np.hstack((output0,output1))
print(y.shape)

(5717, 8192, 2)
(11502, 8192, 2)
(11502,)


## LOG

In [6]:
acc0=np.empty([20])
acc1=np.empty([20])
for k in range(20):
    st,ed=700,3700
    x_train, x_test, y_train, y_test = train_test_split(x[:,st:ed,:], y, test_size=0.30)
    x_train=np.mean(x_train,axis=1)
    x_test=np.mean(x_test,axis=1)
    print(x_train[1])
    classifier = LogisticRegression(random_state = 42, max_iter=500)
    classifier.fit(x_train, y_train)
    # acc=classifier.score(x_test,y_test)
    pred = classifier.predict(x_test)
    count1_acc,count0_acc=0,0
    count1,count0=0,0
    for i in range(pred.shape[0]):
        if y_test[i]==1:
            count1_acc+=1
            if pred[i]>=0.5:
                count1+=1
        else:
            count0_acc+=1
            if pred[i]<0.5:
                count0+=1
    acc1[k]=count1/count1_acc
    acc0[k]=count0/count0_acc
    print(acc0[k],acc1[k])
    

[-2.64894907 -5.77228034]
1.0 0.9977181973759269
[-0.65009494 -3.27106301]
0.9994212962962963 0.9988392338943702
[0.36290237 3.66032719]
1.0 0.996594778660613
[-4.02106677 -3.98120437]
0.9994256174612292 0.9982456140350877
[-2.51753645 -4.22032726]
1.0 0.9971735443753533
[-0.77569505 -4.64115955]
0.9994107248084856 0.9977194982896237
[-0.24068148  4.24751517]
1.0 0.9988694177501414
[-3.27361078  3.28915899]
1.0 0.9977051061388411
[-2.57257094 -4.31181273]
1.0 0.997709049255441
[-0.27716135  4.05211584]
1.0 0.9988512349224583
[0.63247846 3.28109393]
0.9994266055045872 0.9994141769185706
[1.45347951 3.86285744]
1.0 0.9982164090368609
[-0.96395292  5.3878094 ]
1.0 0.9988472622478386
[ 1.68244709 -0.00619227]
1.0 0.997716894977169
[2.73755469 2.00981868]
1.0 0.9988610478359908
[1.09391547 0.90670041]
1.0 0.9971264367816092
[0.48870333 1.25189467]
1.0 0.9988485895221646
[-1.39206942 -2.35062457]
1.0 0.9994356659142212
[-0.57230667  2.08562232]
1.0 0.9977116704805492
[-0.13647547 -3.75345765

In [7]:
print(acc0.mean()*100,acc0.std()*100)
print(acc1.mean()*100,acc1.std()*100)

99.98551085946823 0.02509758363578518
99.81936363138901 0.07670899512096716


## SVM

In [8]:
acc0=np.empty([20])
acc1=np.empty([20])
for k in range(20):
    st,ed=700,3700
    x_train, x_test, y_train, y_test = train_test_split(x[:,st:ed,:], y, test_size=0.30)
    x_train=np.mean(x_train,axis=1)
    x_test=np.mean(x_test,axis=1)
    print(x_train[1])
    model=SVC(decision_function_shape='ovo')
    model.fit(x_train,y_train)
    pred=model.predict(x_test)
    count1_acc,count0_acc=0,0
    count1,count0=0,0
    for i in range(pred.shape[0]):
        if y_test[i]==1:
            count1_acc+=1
            if pred[i]>=0.5:
                count1+=1
        else:
            count0_acc+=1
            if pred[i]<0.5:
                count0+=1
    acc1[k]=count1/count1_acc
    acc0[k]=count0/count0_acc
    print(acc0[k],acc1[k])

[-2.10616776  4.03641644]
0.9988228369629194 0.9965753424657534
[-2.28238619 -2.36059849]
1.0 0.9954776710005653
[-2.0349411   4.26530079]
1.0 0.9971181556195965
[0.51154913 0.68102357]
1.0 0.9965217391304347
[-1.05482952 -1.51327418]
0.9982886480319453 0.9964664310954063
[-1.18606353  3.27715055]
0.9994121105232217 0.9982857142857143
[-2.97937389 -2.71161517]
0.9994093325457767 0.9988623435722411
[-0.66761739 -2.01621755]
0.9988324576765908 0.997698504027618
[-2.99926177 -4.70382431]
0.9994186046511628 0.997689196995956
[-2.4845227   0.65411377]
1.0 0.9977077363896848
[-3.22990935 -2.98559504]
1.0 0.9965297860034702
[-4.72002241 -6.34915434]
0.9994182664339732 0.9948036951501155
[-1.29350569 -3.8427211 ]
0.9994162288382954 0.996547756041427
[-3.28151127 -5.89765297]
1.0 0.9965772960638904
[-1.93805056  1.69669129]
0.9994138335287222 0.9959885386819485
[-2.62993518 -3.14255927]
1.0 0.9965136548518303
[-3.31895529 -2.59852036]
0.9994138335287222 0.9959885386819485
[2.79412573 1.85314533

In [9]:
print(acc0.mean()*100,acc0.std()*100)
print(acc1.mean()*100,acc1.std()*100)

99.9504996809524 0.049345607610346745
99.6836902054348 0.09692737766503662


## GMM

In [15]:
acc0=np.empty([20])
acc1=np.empty([20])
for k in range(20):
    st,ed=700,3700
    x_train, x_test, y_train, y_test = train_test_split(x[:,st:ed,:], y, test_size=0.30)
    x_train=np.mean(x_train,axis=1)
    x_test=np.mean(x_test,axis=1)
    print(x_train[1])
    n_components = 2
    gmm = GaussianMixture(n_components=n_components)
    gmm.fit(x_train)
    pred = gmm.predict(x_test)
    count1_acc,count0_acc=0,0
    count1,count0=0,0
    for i in range(pred.shape[0]):
        if y_test[i]==1:
            count1_acc+=1
            if pred[i]>=0.5:
                count1+=1
        else:
            count0_acc+=1
            if pred[i]<0.5:
                count0+=1
    acc1[k]=count1/count1_acc
    acc0[k]=count0/count0_acc
    if acc1[k]<0.50:
        acc1[k]=1-acc1[k]
        acc0[k]=1-acc0[k]
    print(acc0[k],acc1[k])

[-3.17371638 -5.55792549]
0.9988392338943702 0.9918981481481481
[3.35539578 4.11289461]
0.9982886480319453 0.991166077738516
[-2.5785016   1.80944177]
0.9988564894225271 0.9976498237367802
[-2.93183379 -3.35527308]
0.9988551803091014 0.994131455399061
[3.35699463 4.60678141]
0.9988425925925926 0.9941961694718514
[-3.05595026 -5.13642338]
1.0 0.9971671388101983
[ 1.39730478 -1.00070114]
0.9994327850255247 0.9940758293838863
[-0.82374917 -3.85338582]
0.9994145199063232 0.993689041881813
[-1.83211503 -3.44972227]
0.9976798143851509 0.9936305732484076
[ 0.2475717  -0.21128473]
0.9982507288629737 0.9930875576036866
[-0.26890501  3.06354132]
0.9976918638199653 0.9965075669383003
[-1.140835    0.66797574]
1.0 0.9953542392566783
[-4.82713628 -2.66268746]
0.9994242947610823 0.9941656942823804
[-1.08581297  2.59701323]
0.9988269794721407 0.995418098510882
[1.07906583 4.93141415]
0.9988372093023256 0.9901790872328134
[-4.97210903 -2.84488757]
0.997673065735893 0.9919168591224018
[-0.8944778   4.3

In [16]:
print(acc0.mean()*100,acc0.std()*100)
print(acc1.mean()*100,acc1.std()*100)

99.8839955236359 0.06875403024764856
99.38064442243568 0.19093849685579975


## NN

In [12]:
def arch():
    model=Sequential()
    model.add(Input(shape=(2), name='Input Layer'))
    model.add(Dense(8,activation='relu', name='HiddenLayer1'))
    model.add(Dense(4,activation='relu', name='HiddenLayer2'))
    model.add(Dense(1,activation='sigmoid', name='OuputLayer'))
    return model

In [13]:
acc0=np.empty([20])
acc1=np.empty([20])
for k in range(20):
    st,ed=700,3700
    x_train, x_test, y_train, y_test = train_test_split(x[:,st:ed,:], y, test_size=0.30)
    x_train=np.mean(x_train,axis=1)
    x_test=np.mean(x_test,axis=1)
    model=arch()
    opt=SGD( learning_rate=0.01, momentum=0.9)
    model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, restore_best_weights=True)
    H=model.fit(x_train,y_train,
          validation_split=0.2,
          epochs=50,batch_size=64,
        #   callbacks=[early_stopping],
          verbose=1)
    pred=model.predict(x_test)
    count1_acc,count0_acc=0,0
    count1,count0=0,0
    for i in range(pred.shape[0]):
        if y_test[i]==1:
            count1_acc+=1
            if pred[i]>=0.5:
                count1+=1
        else:
            count0_acc+=1
            if pred[i]<0.5:
                count0+=1
    acc1[k]=count1/count1_acc
    acc0[k]=count0/count0_acc
    print(acc0[k],acc1[k])
    
    

Epoch 1/50


2023-11-06 10:15:30.782648: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


101/101 [==============================] - 1s 3ms/step - loss: 0.9765 - accuracy: 0.3980 - val_loss: 0.6951 - val_accuracy: 0.4960
Epoch 2/50
101/101 [==============================] - 0s 2ms/step - loss: 0.5581 - accuracy: 0.7045 - val_loss: 0.4718 - val_accuracy: 0.9019
Epoch 3/50
101/101 [==============================] - 0s 2ms/step - loss: 0.4172 - accuracy: 0.9547 - val_loss: 0.3758 - val_accuracy: 0.9745
Epoch 4/50
101/101 [==============================] - 0s 2ms/step - loss: 0.3442 - accuracy: 0.9854 - val_loss: 0.3198 - val_accuracy: 0.9851
Epoch 5/50
101/101 [==============================] - 0s 2ms/step - loss: 0.2993 - accuracy: 0.9873 - val_loss: 0.2825 - val_accuracy: 0.9870
Epoch 6/50
101/101 [==============================] - 0s 2ms/step - loss: 0.2676 - accuracy: 0.9887 - val_loss: 0.2543 - val_accuracy: 0.9876
Epoch 7/50
101/101 [==============================] - 0s 2ms/step - loss: 0.2425 - accuracy: 0.9894 - val_loss: 0.2313 - val_accuracy: 0.9901
Epoch 8/50
101/10

In [14]:
print(acc0.mean()*100,acc0.std()*100)
print(acc1.mean()*100,acc1.std()*100)

99.9651981725099 0.03839305457651255
99.78674373033887 0.10338439301630462
